# Models

This page discusses details related to the MLFlow model component, which is responsible for model versioning and deployment.

In [1]:
import mlflow
import mlflow.pyfunc.utils

!rm -rf /tmp/models_set_up
mlflow.set_tracking_uri("file:///tmp/models_set_up")

## PyFunc

The `mlflow.pyfunc` module implements the mlflow features that allow you to build flawour-agnostic models. It is ideal for highly specific approaches that don't fit within the boundaries of any popular framework.

---

The following cell defines the `example_model` function, which performs a computation intended to imitate a imtate model. Log it and register it in the `mlflow` registry.

In [2]:
@mlflow.pyfunc.utils.pyfunc
def example_model(model_intput: list[float]) -> list[float]:
    return list(map(lambda x: x**2, model_intput))

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        name="model",
        python_model=example_model,
        registered_model_name="pyfunc_model",
        pip_requirements=[]
    )

/home/user/.virtualenvironments/python/lib/python3.13/site-packages/mlflow/pyfunc/utils/data_validation.py:155: FutureWarning: Model's `predict` method contains invalid parameters: {'model_intput'}. Only the following parameter names are allowed: context, model_input, and params. Note that invalid parameters will no longer be permitted in future versions.
  param_names = _check_func_signature(func, "predict")
Successfully registered model 'pyfunc_model'.
Created version '1' of model 'pyfunc_model'.


The following cell shows the code that loads model, and uses the model.

In [3]:
model = mlflow.pyfunc.load_model("models:/pyfunc_model/1")
model.predict([1., 2., 3.])

[1.0, 4.0, 9.0]

The results correspond to the calculation specified in the `example_model` function.